## Controlling the AAS WorldWide Telescope Windows client using pywwt

This notebook demonstrates how to use the AAS WorldWide Telescope Windows client using pywwt. You can find out more about this in the [documentation](http://pywwt.readthedocs.io/en/latest/windows.html).

First, we'll import the `WWTWindowsClient` class, as well as [numpy](http://www.numpy.org) and [h5py](http://www.h5py.org), which we'll need for this example. We will also import [astropy.units](https://docs.astropy.org/en/stable/units/index.html) to handle unit conversions

In [ ]:
import numpy as np
import h5py
from astropy import units as u
from pywwt.windows import WWTWindowsClient

Next, we'll connect to WorldWide Telescope:

In [ ]:
wwt = WWTWindowsClient()

Now, we'll create a layer which we'll put data in later. Since this is tracer particle data from a radio minihalo simulation of a galaxy cluster core, we'll put the layer in the `"Sky"` frame and name it `"minihalo"`. We also need to specify the fields that will go into the layer, which in this case are just the spherical coordinates of the points and the color of the point. 

In [ ]:
layer = wwt.new_layer("Sky","minihalo",["RA","DEC","ALT","color"])

We need to set the properties of this layer as well. We will construct a `props_dict` to hold the layer properties we want. A summary:

- The coordinates are spherical.
- The size of the point is relative to the screen.
- The points have all the same scale.
- The scale of the points is 16. 
- The points are lit on all sides. 
- This is not a time series dataset. 
- The "altitude" coordinate (or radius) is in megaparsecs.
- The units of the RA coordinate are in degrees. 

In [ ]:
props_dict = {"CoordinatesType":"Spherical",
              "MarkerScale":"Screen",
              "PointScaleType":"Constant",
              "ScaleFactor":"16",
              "ShowFarSide":"True",
              "TimeSeries":"False",
              "AltUnit":"MegaParsecs",
              "RaUnits":"Degrees"}
layer.set_properties(props_dict)

Though it probably already is, this call activates the layer in the view: 

In [ ]:
layer.activate()

Now we have to do some work to get the data out of the HDF5 file it's contained in. The data consists of a set of points, with cartesian coordinates in centimeters, and radio emissivity in cgs units. We'll map the radio emissivity to a color map from Matplotlib, and convert the coordinates to spherical. We'll put all of this in a dict, `data`. 

In [ ]:
from pywwt.windows import map_array_to_colors, convert_xyz_to_spherical
fn = "radio_halo_1kpc_hdf5_part_0200_reduced.h5"
f = h5py.File(fn, "r")
cm_per_mpc = u.Mpc.to(u.cm)
x = f["x"][:] / cm_per_mpc # The coordinates in the file are in cm, this converts them to Mpc
y = f["y"][:] / cm_per_mpc
z = f["z"][:] / cm_per_mpc
c = f["radio"][:]
color = map_array_to_colors(c, "spectral", scale="log", vmin=1.0e-40, vmax=4.0e-23)
data = convert_xyz_to_spherical(x, y, z)
data["color"] = color
f.close()

Now we add this data in. We set `purge_all=True` to eliminate the data already in the layer (though it was empty so it's superfluous), and we set the `fly_to` parameter to fly to a particular location and zoom setting relative to the `"Sky"` frame:

- Latitude: 48 degrees
- Longitude: -12 degress
- Zoom: $6 \times 10^{11}$
- Rotation: 0 radians
- Angle: 0 radians

In [ ]:
layer.update(data=data, purge_all=True, fly_to=[48.,-12.,6.0e11,0.,0.])

Just as a check, we can get the state of the current view (after the fly-to stops) and see that it matches up with the coordinates of our `fly_to` parameter:

In [ ]:
wwt.get_state()